## Simplest

In [1]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

model = gp.Model()
model.params.NonConvex = 2

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Set parameter NonConvex to value 2


here need $P_{ZAMY}$

In [2]:
P_AMY_giv_Z = {(0, 0, 0, 0): 0.336, (0, 1, 0, 0): 0.084, (1, 0, 0, 0): 0.056, (1, 1, 0, 0): 0.224, (0, 0, 0, 1): 0.096, (0, 1, 0, 1): 0.024, (1, 0, 0, 1): 0.036, (1, 1, 0, 1): 0.144, (0, 0, 1, 0): 0.144, (0, 1, 1, 0): 0.036, (1, 0, 1, 0): 0.024, (1, 1, 1, 0): 0.096, (0, 0, 1, 1): 0.224, (0, 1, 1, 1): 0.056, (1, 0, 1, 1): 0.084, (1, 1, 1, 1): 0.336}

In [3]:
(cardZ, cardA, cardM, cardY) = (2,2,2,2)

Q_AzAz_MaMa_YmYm = model.addMVar(shape = (cardA, cardA, cardM, cardM, cardY, cardY), 
                             vtype=GRB.CONTINUOUS, name="Q_(A0,A1,M0,M1,Y0,Y1)", lb=0, ub=1)

In [4]:
# coordinate finder used for P(a,m,y|z) = Σ_(a',m',y')[Q_(a,a',m,m',y,y')]
def Q_pos_from_P(a, m, y, z):
    if z == 0:
        posA0 = a
        posA1 = slice(None)  # Represents ':'
    else:  # z == 1
        posA0 = slice(None)
        posA1 = a
    
    # determining positions for m and y
    if a==0:
        posM0 = m
        posM1 = slice(None)
    elif a==1:
        posM0 = slice(None)
        posM1 = m

    if m==0:
        posY0 = y
        posY1 = slice(None)
    elif m==1:
        posY0 = slice(None)
        posY1 = y

    # Creating a tuple that properly represents the slicing
    return (posA0, posA1, posM0, posM1, posY0, posY1)

# coordinate finder used for P_YM_doA[y,m,a] = Σ_(A0, A1, M_A\not=a, Y_M\not=m)[Q_(a,a',m,m',y,y')]
def do_pos(y,m,a):
    # sliced means iterate through in the sum
    
    # keep M such that M_A=m, slice other
    if a == 0:
        posM0 = m
        posM1 = slice(None)
    elif a == 1:    
        posM0 = slice(None)
        posM1 = m


    if m == 0:
        posY0 = y
        posY1 = slice(None)
    elif m == 1:
        posY0 = slice(None)
        posY1 = y


    # Creating a tuple that properly represents the slicing
    return (slice(None), slice(None), posM0, posM1, posY0, posY1) 

In [5]:
# consistency constraints
for a, m, y, z in itertools.product(range(cardA), range(cardM), range(cardY), range(cardZ)):
    model.addConstr(Q_AzAz_MaMa_YmYm[Q_pos_from_P(a,m,y,z)].sum() 
                    == 
                    P_AMY_giv_Z[a,m,y,z], 
                    name=f"{P_AMY_giv_Z[a,m,y,z]} = P_AMY|Z[{a,m,y}|{z}]")

# seperability constraints
for a0, a1, m0, m1 in itertools.product(range(cardA), range(cardA), range(cardM), range(cardM)):
    model.addConstr(Q_AzAz_MaMa_YmYm[a0, a1, m0, m1, :, :].sum()
                    == 
                    #(Q_AzAz_MaMa_YmYm[a0, a1, :, :, :, :].sum() * Q_AzAz_MaMa_YmYm[:, :, m0, m1, :, :].sum()).sum(),
                    Q_AzAz_MaMa_YmYm[a0, a1, :, :, :, :].sum() * Q_AzAz_MaMa_YmYm[:, :, m0, m1, :, :].sum(), 
                    name=f"Q_A0_A1_M0_M1[{a0},{a1},{m0},{m1}] = Q_A0_A1[{a0},{a1}] * Q_M0_M1[{m0},{m1}]")

# other seperability constraints
for a,y in np.ndindex(cardA, cardY):
    model.addConstr((Q_AzAz_MaMa_YmYm[do_pos(y,0,a)] + Q_AzAz_MaMa_YmYm[do_pos(y,1,a)]).sum() 
                    == 
                    Q_AzAz_MaMa_YmYm[do_pos(y,0,a)].sum() + Q_AzAz_MaMa_YmYm[do_pos(y,1,a)].sum(), 
                    name=f"P_Y_doA[{a},{y}] = P_YM_doA[{y},0,{a}] + P_YM_doA[{y},1,{a}]")

model.update()

In [6]:
def P_Y_doA(y, a):
    return (Q_AzAz_MaMa_YmYm[do_pos(y,0,a)] + Q_AzAz_MaMa_YmYm[do_pos(y,1,a)]).sum()

## Execute:

In [10]:
(y,b) = (1,1)

In [11]:
objective = P_Y_doA(y,b)
model.setObjective(objective, sense=GRB.MINIMIZE)
model.optimize()
minimal = P_Y_doA(y,b).getValue()


objective = P_Y_doA(y,b)
model.setObjective(objective, sense=GRB.MAXIMIZE)
model.optimize()
maximal = P_Y_doA(y,b).getValue()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 64 columns and 128 nonzeros
Model fingerprint: 0xb7576061
Model has 16 quadratic constraints
Variable types: 64 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-02, 3e-01]



Processing MIP start from previous solve: 325 nodes explored in subMIP, total elapsed time 5s
MIP start from previous solve produced solution with objective 0.42 (9.41s)
Loaded MIP start from previous solve with objective 0.42
Processed MIP start in 9.43 seconds (23.99 work units)

Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 8288 rows, 2144 columns, 20767 nonzeros
Presolved model has 2080 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 2144 continuous, 0 integer (0 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e-01   4.837072e+00   0.000000e+00     10s

Root relaxation: cutoff, 15 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         0.42000    0.42000  0.00%     -    9s

Explored 1 nodes (15 simplex

In [12]:
print("min:", minimal, "\nmax:", maximal, "\nrange:", maximal-minimal)

min: 0.41999999999999993 
max: 0.72 
range: 0.30000000000000004
